## Task-1

### Data Exploration and Enrichment

In [5]:
import pandas as pd

df = pd.read_csv("../data/raw/ethiopia_fi_unified_data.csv")
ref = pd.read_csv("../data/raw/reference_codes.csv")
impact_df = pd.read_csv("../data/raw/Impact_sheet.csv")

df.head()
df.info()


<class 'pandas.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   record_id            43 non-null     str    
 1   record_type          43 non-null     str    
 2   category             10 non-null     str    
 3   pillar               33 non-null     str    
 4   indicator            43 non-null     str    
 5   indicator_code       43 non-null     str    
 6   indicator_direction  33 non-null     str    
 7   value_numeric        33 non-null     float64
 8   value_text           10 non-null     str    
 9   value_type           43 non-null     str    
 10  unit                 33 non-null     str    
 11  observation_date     43 non-null     str    
 12  period_start         10 non-null     str    
 13  period_end           10 non-null     str    
 14  fiscal_year          43 non-null     str    
 15  gender               43 non-null     str    
 16  loc

#### Confirm unified Schema

In [6]:
df.columns
df['record_type'].unique()

<StringArray>
['observation', 'target', 'event']
Length: 3, dtype: str

#### Check record counts 

In [7]:
df['record_type'].value_counts()

record_type
observation    30
event          10
target          3
Name: count, dtype: int64

#### Pillar assignment logic

In [14]:
df.groupby(['record_type', 'pillar']).size()

record_type  pillar       
observation  ACCESS           14
             AFFORDABILITY     1
             GENDER            4
             USAGE            11
target       ACCESS            2
             GENDER            1
dtype: int64

#### temporal range

In [ ]:
df['observation_date'].min(), df['observation_date'].max()

('2014-12-31', '2030-12-31')

#### Indicator coverage

In [16]:
df[df.record_type=="observation"]['indicator_code'].value_counts()

indicator_code
ACC_OWNERSHIP         6
ACC_FAYDA             3
ACC_MM_ACCOUNT        2
ACC_4G_COV            2
USG_P2P_COUNT         2
GEN_GAP_ACC           2
ACC_MOBILE_PEN        1
USG_P2P_VALUE         1
USG_ATM_COUNT         1
USG_ATM_VALUE         1
USG_CROSSOVER         1
USG_TELEBIRR_USERS    1
USG_TELEBIRR_VALUE    1
USG_MPESA_USERS       1
USG_MPESA_ACTIVE      1
USG_ACTIVE_RATE       1
AFF_DATA_INCOME       1
GEN_MM_SHARE          1
GEN_GAP_MOBILE        1
Name: count, dtype: int64

#### Events inventory 

In [17]:
df[df.record_type=="event"][['indicator','category','observation_date']]

,indicator,category,observation_date
33,Telebirr Launch,product_launch,2021-05-17
34,Safaricom Ethiopia Commercial Launch,market_entry,2022-08-01
35,M-Pesa Ethiopia Launch,product_launch,2023-08-01
36,Fayda Digital ID Program Rollout,infrastructure,2024-01-01
37,Foreign Exchange Liberalization,policy,2024-07-29
38,P2P Transaction Count Surpasses ATM,milestone,2024-10-01
39,M-Pesa EthSwitch Integration,partnership,2025-10-27
40,EthioPay Instant Payment System Launch,infrastructure,2025-12-18
41,NFIS-II Strategy Launch,policy,2021-09-01
42,Safaricom Ethiopia Price Increase,pricing,2025-12-15


#### Existing Impact links 

In [18]:
df[df.record_type=="impact_link"]

,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes


#### Explore Existing Impacts

In [8]:
impact_df['record_type'].value_counts()

record_type
impact_link    14
Name: count, dtype: int64

#### Merge impact sheet with main dataset 

In [9]:
# If your main df has the impact_links sheet as well
df_combined = pd.concat([df[df.record_type=="impact_link"], impact_df], ignore_index=True)

#### Count impacts by pillar

In [10]:
impact_df['pillar'].value_counts()


pillar
USAGE            6
ACCESS           4
AFFORDABILITY    3
GENDER           1
Name: count, dtype: int64

#### Count impacts by evidence type

In [11]:
impact_df['evidence_basis'].value_counts()

evidence_basis
literature     7
empirical      6
theoretical    1
Name: count, dtype: int64

#### Lag distribution

In [12]:
impact_df['lag_months'].describe()

count    14.000000
mean      8.428571
std       7.562196
min       1.000000
25%       3.000000
50%       6.000000
75%      12.000000
max      24.000000
Name: lag_months, dtype: float64

#### Impact Magnitude

In [13]:
impact_df['impact_magnitude'].value_counts()

impact_magnitude
medium    8
high      5
low       1
Name: count, dtype: int64

In [22]:
df["record_type"].value_counts()
impact_df["record_type"].value_counts()

record_type
impact_link    14
Name: count, dtype: int64

### Adding new Observations, Events, Impact_links and Updating reference codes

In [4]:
import pandas as pd

data_path = "../data/raw/ethiopia_fi_unified_data.csv"
impact_path = "../data/raw/impact_sheet.csv"  # if separate
ref_path = "../data/raw/reference_codes.csv"

df = pd.read_csv(data_path)
impact_df = pd.read_csv(impact_path)
ref = pd.read_csv(ref_path)


#### Add new Observations

In [5]:
new_observation = {
    "record_id": "OBS_0031",
    "record_type": "observation",
    "category": None,
    "pillar": "ACCESS",
    "indicator": "Population covered by 4G network",
    "indicator_code": "ACC_4G_COV",
    "indicator_direction": "positive",
    "value_numeric": 55,
    "value_text": None,
    "value_type": "percentage",
    "unit": "%",
    "observation_date": "2023-12-31",
    "period_start": None,
    "period_end": None,
    "fiscal_year": 2023,
    "gender": "all",
    "location": "national",
    "region": None,
    "source_name": "GSMA Mobile Connectivity Index",
    "source_type": "secondary",
    "source_url": "https://www.gsma.com",
    "confidence": "medium",
    "related_indicator": None,
    "relationship_type": None,
    "impact_direction": None,
    "impact_magnitude": None,
    "impact_estimate": None,
    "lag_months": None,
    "evidence_basis": None,
    "comparable_country": None,
    "collected_by": "Abrham Molla",
    "collection_date": "2026-01-30",
    "original_text": "4G population coverage in Ethiopia reached approximately 55% in 2023.",
    "notes": "Infrastructure proxy for digital payment usage growth"
}
df = pd.concat([df, pd.DataFrame([new_observation])], ignore_index=True)
print('successfully added new observation')

successfully added new observation


In [6]:
new_observation = {
    "record_id": "OBS_0032",
    "parent_id": "",
    "record_type": "observation",
    "category": "",
    "pillar": "USAGE",
    "indicator": "Share of government payments made digitally",
    "indicator_code": "USG_G2P_DIGITIZED",
    "indicator_direction": "higher_better",
    "value_numeric": 18,
    "value_text": "",
    "value_type": "percentage",
    "unit": "%",
    "observation_date": "2024-12-31",
    "period_start": "",
    "period_end": "",
    "fiscal_year": 2024,
    "gender": "all",
    "location": "national",
    "region": "",
    "source_name": "National Bank of Ethiopia",
    "source_type": "regulator",
    "source_url": "https://nbe.gov.et/",
    "confidence": "medium",
    "related_indicator": "",
    "relationship_type": "",
    "impact_direction": "",
    "impact_magnitude": "",
    "impact_estimate": "",
    "lag_months": "",
    "evidence_basis": "",
    "comparable_country": "",
    "collected_by": "Abrham Molla",
    "collection_date": "2026-01-30",
    "original_text": "An estimated 18% of government-to-person payments were processed digitally in 2024.",
    "notes": "Digitized G2P payments are a strong driver of habitual digital payment usage."
}
df = pd.concat([df, pd.DataFrame([new_observation])], ignore_index=True)
df.tail()


,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes,parent_id
40,EVT_0008,event,infrastructure,NaN,EthioPay Instant Payment System Launch,EVT_ETHIOPAY,NaN,NaN,Launched,categorical,...,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,National real-time payment system,NaN,NaN
41,EVT_0009,event,policy,NaN,NFIS-II Strategy Launch,EVT_NFIS2,NaN,NaN,Launched,categorical,...,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,5-year national financial inclusion strategy,NaN,NaN
42,EVT_0010,event,pricing,NaN,Safaricom Ethiopia Price Increase,EVT_SAFCOM_PRICE,NaN,NaN,Implemented,categorical,...,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Data and voice prices increased 20-82%,NaN,NaN
43,OBS_0031,observation,None,ACCESS,Population covered by 4G network,ACC_4G_COV,positive,55.0,None,percentage,...,None,None,None,None,None,Abrham Molla,2026-01-30,4G population coverage in Ethiopia reached app...,Infrastructure proxy for digital payment usage...,NaN
44,OBS_0032,observation,,USAGE,Share of government payments made digitally,USG_G2P_DIGITIZED,higher_better,18.0,,percentage,...,,,,,,Abrham Molla,2026-01-30,An estimated 18% of government-to-person payme...,Digitized G2P payments are a strong driver of ...,


#### Add new Events

In [7]:
new_event = {
    "record_id": "EVT_0011",
    "record_type": "event",
    "category": "policy",
    "pillar": None,  # IMPORTANT
    "indicator": "Government Wage Digitization Initiative",
    "indicator_code": None,
    "indicator_direction": None,
    "value_numeric": None,
    "value_text": None,
    "value_type": None,
    "unit": None,
    "observation_date": "2022-01-01",
    "period_start": None,
    "period_end": None,
    "fiscal_year": 2022,
    "gender": "all",
    "location": "national",
    "region": None,
    "source_name": "Ministry of Finance Ethiopia",
    "source_type": "government",
    "source_url": None,
    "confidence": "medium",
    "related_indicator": None,
    "relationship_type": None,
    "impact_direction": None,
    "impact_magnitude": None,
    "impact_estimate": None,
    "lag_months": None,
    "evidence_basis": None,
    "comparable_country": None,
    "collected_by": "Abrham Molla",
    "collection_date": "2026-01-30",
    "original_text": None,
    "notes": "Expected to increase DFS usage via recurring payments"
}

df = pd.concat([df, pd.DataFrame([new_event])], ignore_index=True)

In [8]:
new_event = {
    "record_id": "EVT_0011",
    "parent_id": "",
    "record_type": "event",
    "category": "policy",
    "pillar": "",
    "indicator": "Expansion of digital government payments (PSNP & pensions)",
    "indicator_code": "",
    "indicator_direction": "",
    "value_numeric": "",
    "value_text": "",
    "value_type": "",
    "unit": "",
    "observation_date": "2023-01-01",
    "period_start": "",
    "period_end": "",
    "fiscal_year": 2023,
    "gender": "all",
    "location": "national",
    "region": "",
    "source_name": "Ministry of Finance Ethiopia",
    "source_type": "policy",
    "source_url": "",
    "confidence": "medium",
    "related_indicator": "",
    "relationship_type": "",
    "impact_direction": "",
    "impact_magnitude": "",
    "impact_estimate": "",
    "lag_months": "",
    "evidence_basis": "",
    "comparable_country": "",
    "collected_by": "Abrham Molla",
    "collection_date": "2026-01-30",
    "original_text": "The government began expanding digital delivery of social protection and pension payments.",
    "notes": "Recurring government payments create sustained digital account usage."
}
df = pd.concat([df, pd.DataFrame([new_event])], ignore_index=True)
df.tail()

,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes,parent_id
42,EVT_0010,event,pricing,NaN,Safaricom Ethiopia Price Increase,EVT_SAFCOM_PRICE,NaN,NaN,Implemented,categorical,...,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Data and voice prices increased 20-82%,NaN,NaN
43,OBS_0031,observation,None,ACCESS,Population covered by 4G network,ACC_4G_COV,positive,55.0,None,percentage,...,None,None,None,None,None,Abrham Molla,2026-01-30,4G population coverage in Ethiopia reached app...,Infrastructure proxy for digital payment usage...,NaN
44,OBS_0032,observation,,USAGE,Share of government payments made digitally,USG_G2P_DIGITIZED,higher_better,18.0,,percentage,...,,,,,,Abrham Molla,2026-01-30,An estimated 18% of government-to-person payme...,Digitized G2P payments are a strong driver of ...,
45,EVT_0011,event,policy,None,Government Wage Digitization Initiative,None,None,None,None,None,...,None,None,None,None,None,Abrham Molla,2026-01-30,None,Expected to increase DFS usage via recurring p...,NaN
46,EVT_0011,event,policy,,Expansion of digital government payments (PSNP...,,,,,,...,,,,,,Abrham Molla,2026-01-30,The government began expanding digital deliver...,Recurring government payments create sustained...,


#### Add new Impact_links

In [9]:
new_impact = {
    "record_id": "IMP_0015",
    "parent_id": "EVT_0011",
    "record_type": "impact_link",
    "category": None,
    "pillar": "USAGE",
    "indicator": "Wage digitization effect on digital payments",
    "indicator_code": None,
    "indicator_direction": None,
    "value_numeric": None,
    "value_text": None,
    "value_type": None,
    "unit": None,
    "observation_date": "2022-01-01",
    "period_start": None,
    "period_end": None,
    "fiscal_year": 2022,
    "gender": "all",
    "location": "national",
    "region": None,
    "source_name": None,
    "source_type": None,
    "source_url": None,
    "confidence": "medium",
    "related_indicator": "USG_DIGITAL_PAYMENT",
    "relationship_type": "direct",
    "impact_direction": "increase",
    "impact_magnitude": "medium",
    "impact_estimate": None,
    "lag_months": 6,
    "evidence_basis": "literature",
    "comparable_country": "Ghana",
    "collected_by": "Abrham Molla",
    "collection_date": "2026-01-30",
    "original_text": "Government payroll digitization increased digital payment usage in Ghana.",
    "notes": "Recurring payments create habitual usage"
}

impact_df = pd.concat([impact_df, pd.DataFrame([new_impact])], ignore_index=True)

In [10]:
new_impact_link = {
    "record_id": "IMP_0015",
    "parent_id": "EVT_0011",
    "record_type": "impact_link",
    "category": "",
    "pillar": "USAGE",
    "indicator": "G2P digitization effect on digital payment usage",
    "indicator_code": "",
    "indicator_direction": "",
    "value_numeric": "",
    "value_text": "",
    "value_type": "",
    "unit": "",
    "observation_date": "2023-01-01",
    "period_start": "",
    "period_end": "",
    "fiscal_year": "",
    "gender": "all",
    "location": "national",
    "region": "",
    "source_name": "",
    "source_type": "",
    "source_url": "",
    "confidence": "medium",
    "related_indicator": "USG_DIGITAL_PAYMENT",
    "relationship_type": "direct",
    "impact_direction": "increase",
    "impact_magnitude": "medium",
    "impact_estimate": 8,
    "lag_months": 9,
    "evidence_basis": "literature",
    "comparable_country": "Kenya",
    "collected_by": "Abrham Molla",
    "collection_date": "2026-01-30",
    "original_text": "Digitized government payments increased digital payment usage by 5–10pp in comparable countries.",
    "notes": "G2P payments act as a first and recurring digital finance use case."
}
impact_df = pd.concat([impact_df, pd.DataFrame([new_impact_link])], ignore_index=True)
impact_df.tail()

,record_id,parent_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
11,IMP_0012,EVT_0007,impact_link,NaN,USAGE,M-Pesa Interop effect on P2P Count,NaN,NaN,10.0,NaN,...,increase,medium,10.0,3,literature,Tanzania,Example_Trainee,2025-01-20,NaN,Cross-platform transactions now possible
12,IMP_0013,EVT_0008,impact_link,NaN,USAGE,EthioPay effect on P2P Count,NaN,NaN,15.0,NaN,...,increase,medium,15.0,6,literature,India,Example_Trainee,2025-01-20,NaN,India UPI showed +25% volume increase
13,IMP_0014,EVT_0010,impact_link,NaN,AFFORDABILITY,Safaricom Price Hike effect on Data Affordability,NaN,NaN,10.0,NaN,...,increase,low,10.0,1,empirical,NaN,Example_Trainee,2025-01-20,NaN,Direct price increase (may be offset by switch...
14,IMP_0015,EVT_0011,impact_link,None,USAGE,Wage digitization effect on digital payments,None,None,None,None,...,increase,medium,None,6,literature,Ghana,Abrham Molla,2026-01-30,Government payroll digitization increased digi...,Recurring payments create habitual usage
15,IMP_0015,EVT_0011,impact_link,,USAGE,G2P digitization effect on digital payment usage,,,,,...,increase,medium,8,9,literature,Kenya,Abrham Molla,2026-01-30,Digitized government payments increased digita...,G2P payments act as a first and recurring digi...


#### Define new reference code entries

In [11]:
import pandas as pd

# Load existing reference codes
ref_path = "../data/raw/reference_codes.csv"
ref_df = pd.read_csv(ref_path)

# New reference codes to add
new_reference_codes = [
    {
        "code": "USG_G2P_DIGITIZED",
        "type": "indicator",
        "description": "Share of government-to-person payments made digitally",
        "pillar": "USAGE",
        "notes": "Used to track digitization of public transfers and pensions"
    },
    {
        "code": "EVT_G2P_DIGITAL_EXPANSION",
        "type": "event",
        "description": "Expansion of digital government payments (PSNP and pensions)",
        "pillar": "",
        "notes": "Policy-driven expansion of recurring G2P digital payments"
    },
    {
        "code": "IMP_G2P_USAGE_EFFECT",
        "type": "impact_link",
        "description": "Impact of G2P digitization on digital payment usage",
        "pillar": "USAGE",
        "notes": "Estimated using comparable country evidence (Kenya)"
    }
]


In [12]:
new_ref_df = pd.DataFrame(new_reference_codes)

# Only add codes that do not already exist
ref_df_updated = pd.concat(
    [
        ref_df,
        new_ref_df[~new_ref_df["code"].isin(ref_df["code"])]
    ],
    ignore_index=True
)


In [13]:
print(ref_df_updated.tail(5))
# Save updated reference codes
ref_df_updated.to_csv("../data/processed/updated_reference_codes.csv", index=False)
print("Updated reference codes saved successfully.")


             field                       code  \
69  evidence_basis                theoretical   
70  evidence_basis                     expert   
71             NaN          USG_G2P_DIGITIZED   
72             NaN  EVT_G2P_DIGITAL_EXPANSION   
73             NaN       IMP_G2P_USAGE_EFFECT   

                                          description   applies_to  \
69                           Based on economic theory  impact_link   
70                             Domain expert judgment  impact_link   
71  Share of government-to-person payments made di...          NaN   
72  Expansion of digital government payments (PSNP...          NaN   
73  Impact of G2P digitization on digital payment ...          NaN   

           type pillar                                              notes  
69          NaN    NaN                                                NaN  
70          NaN    NaN                                                NaN  
71    indicator  USAGE  Used to track digitization of pu

In [14]:
df.to_csv("../data/processed/ethiopia_fi_unified_data_enriched.csv", index=False)
impact_df.to_csv("../data/processed/impact_links_enriched.csv", index=False)
